# AOC 2023

https://adventofcode.com/2023

In [32]:
from typing import List
from collections import Counter, abc

## Utils

In [87]:
class Parser:
    @staticmethod
    def _read_file(filename, pp) -> List[str]:
        return [pp(l.strip()) for l in open(filename, "r")]

    @staticmethod
    def _read_test_input(string, pp) -> List[str]:
        return [pp(l.strip()) for l in string.split("\n") if len(l.strip()) > 0]
    
    @staticmethod
    def parse_input(sample_str, filename, pp=lambda x: x):
        return Parser._read_test_input(sample_str, pp), Parser._read_file(filename, pp)


class Grid(abc.Mapping):
    """
        2D-grid keyed by (row, col) and value is content
    """
    def __init__(self, lines: List[str]):
        self._grid = {(r, c): v.strip()
                        for r, line in enumerate(lines)
                        for c, v in enumerate(line)}
        self.directions = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
        self.n_cols = len(lines[0])
        self.n_rows = len(lines)

    def valid(self, point):
        r, c = point
        return  (r >= 0 and r < (self.n_rows) and
                 c >= 0 and c < (self.n_cols))

    def __getitem__(self, point):
        return self._grid[point[0], point[1]]

    def __iter__(self):
        return iter(self._grid)

    def __len__(self):
        return self.n_rows * self.n_cols
        
    def neighbors(self, cell):
        r, c = cell
        return [
            ((r + rd), (c + cd))
            for rd, cd in self.directions
                if self.valid(((r + rd), (c + cd)))

        ]

## Day 1
    

### Part 1

Find the first and last digit.

In [211]:
sample_str = """
            1abc2
            pqr3stu8vwx
            a1b2c3d4e5f
            treb7uchet
            """

sample_input, file_input = Parser.parse_input(sample_str, "data/day1.txt")

In [212]:
def d1p1(test=False):
    input = sample_input if test else file_input
    
    first, last = None, None
    def find_digit(line):
        for c in line:
            if c.isdigit():
                return c
        return None

    results = []
    for line in input:
        first, last = find_digit(line), find_digit(line[::-1])
        results.append(int(first + last))
        
    return sum(results)

In [309]:
assert day1(True) == 142
day1()

54644

### Part 2

Since we know that digit names are of length 3, 4, or 5 - we can check if a length k digit name matches from a given index. Do this if its not already a digit.

In [311]:
s2d = {
    "one": "1",
    "two": "2",
    "three": "3",
    "four": "4",
    "five": "5",
    "six": "6",
    "seven": "7",
    "eight": "8",
    "nine": "9"
}
s2d_rev = {k[::-1]: v for k, v in s2d.items()}

sample_str = """
    two1nine
    eightwothree
    abcone2threexyz
    xtwone3four
    4nineeightseven2
    zoneight234
    7pqrstsixteen
"""
sample_input, file_input = Parser.parse_input(sample_str, "data/day1.txt")

def valid_digitname(str, start, valid_s2d):
    # Is there a valid digitname from start index in the input string
    
    ends = [3, 4, 5]
    # Possible lengths for valid digit names
    ends = [3, 4, 5]
    for e in ends:
        candidate_name = str[start: start+e]
        if candidate_name in valid_s2d:
            return valid_s2d[candidate_name]
    return None

def find_digit(line, valid_s2d):
        for i in range(0, len(line), 1):
            c = line[i]
            if c.isdigit():
                return c
            elif (d := valid_digitname(line, i, valid_s2d)):
                return d
        return None

def d1p2(test=False):
    input = sample_input if test else file_input
    results = []
    for line in input:
        first = find_digit(line, s2d)
        last = find_digit(line[::-1], s2d_rev)
        results.append(int(first + last))
        
    return sum(results)

In [312]:
assert d1p2(True) == 281
d1p2()

53348

## Day 2

### Part 1

This seems like a form of CSP. We have some predicate (valid cubes), for each line - check which lines satisfy the predicate.

In [313]:
sample_str = """
    Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
    Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
    Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
    Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
    Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green
"""

def parse_line(line):
    game, rest = line.split(":")
    game_id = int(game.replace("Game ", "").strip())
    
    games = []
    for g in rest.split(";"):
        counts = Counter()
        for hand in g.split(","):
            n, color = hand.strip().split(" ")
            counts[color.strip()] += int(n.strip())
            
        games.append(counts)
    return game_id, games
            
sample_input, file_input = Parser.parse_input(sample_str, "data/day2.txt", pp=parse_line)

In [314]:
total_cubes = {"red": 12, "green": 13, "blue": 14}
valid_hand = lambda picked: (picked["red"] <= total_cubes["red"] and
    picked["green"] <= total_cubes["green"] and 
    picked["blue"] <= total_cubes["blue"])

def d2p1(test=False):
    input = sample_input if test else file_input
    valid_ids = []
    for game_id, game in input:
        if all([valid_hand(hand) for hand in game]):
            valid_ids.append(game_id)

    return sum(valid_ids)

In [315]:
assert d2p1(True) == 8
d2p1()

2551

### Part 2

In [318]:
def update_min_cubes(candidate, best):
    for c in ["red", "green", "blue"]:
        best[c] = max(candidate[c], best[c])

def game_power(hand):
    return hand["red"] * hand["blue"] * hand["green"]

def d2p2(test=False):
    input = sample_input if test else file_input
    powers = []
    for game_id, game in input:
        min_cubes = {"red": 0, "blue": 0, "green": 0}
        for hand in game:
            update_min_cubes(hand, min_cubes)
        powers.append(game_power(min_cubes))
        
    return sum(powers)

In [319]:
assert d2p2(True) == 2286
d2p2()

62811

## Day 3

### Part 1

The basic idea for this part is:

- For every symbol, see if you can find a neighbor digit.
    - If yes, expand left and right to find the full number
    - Mark the indexes that contain the number and don't double count


In [44]:
sample_str = """
    467..114..
    ...*......
    ..35..633.
    ......#...
    617*......
    .....+.58.
    ..592.....
    ......755.
    ...$.*....
    .664.598..
"""

sample_input, file_input = Parser.parse_input(sample_str, "data/day3.txt")

In [56]:
"1".isdigit()

True

In [98]:
def expand_to_number(point, grid):
    # Expand index left and right to find the number
    s = e = point[1]
    while (s >= 0):
        v_s = grid.get((point[0], s - 1), "")
        if (v_s.isdigit()):
            s -= 1
        else:
            break

    while (e < grid.n_cols):
        v_e = grid.get((point[0], e + 1), "")
        if (v_e.isdigit()):
            e += 1
        else:
            break
    return (s, e)

def d3p1(test=False):
    grid = Grid(sample_input) if test else Grid(file_input)
    seen = set()
    parts = []
    symbol_points = [p for p in grid if not grid.get(p).isdigit() and grid.get(p) != "."]
    
    for p in symbol_points:
        neighbors = grid.neighbors(p)
        for n in neighbors:
            if (n not in seen) and (grid.get(n)).isdigit():
                left, right = expand_to_number(n, grid)
                digits = []
                for i in range(left, right+1):
                    p = (n[0], i)
                    seen.add(p)
                    digits.append(grid.get(p))
                parts.append(int("".join(digits)))
    return sum(parts)

In [100]:
assert d3p1(True) == 4361
d3p1()

540025

### Part 2

Similar idea - find gears and multiply. Duplicates don't matter. Handle double counting of neighbors per gear.

In [139]:
def get_gear_ratio(p, grid):
    digit_neighbors = [n for n in grid.neighbors(p) if grid.get(n).isdigit()]
    
    # Prevents double counting of neighbors
    seen = set({})
    parts = []
    
    # Find all numbers surrounding gear.
    for n in digit_neighbors:
        if n in seen:
            continue
            
        digits = []            
        left, right = expand_to_number(n, grid)
        for i in range(left, right+1):
            p = (n[0], i)
            digits.append(grid.get(p))
            seen.add(p)
        
        parts.append(int("".join(digits)))
        
    return parts[0] * parts[1] if len(parts) == 2 else None

def d3p2(test=False):
    grid = Grid(sample_input) if test else Grid(file_input)
    gear_ratios = []
    possible_gears = [p for p in grid if grid.get(p) == "*"]

    for p in possible_gears:
        if (r := get_gear_ratio(p, grid)) is not None:
            gear_ratios.append(r)
            
    return sum(gear_ratios)

In [142]:
assert d3p2(True) == 467835
d3p2()

84584891